# Introduction

## AutoPandas

[AutoPandas](https://autopandas.io) is a input-output example based synthesis engine for the [Pandas](https://pandas.pydata.org) data-analytics library in Python. Users provide input-output pairs (dataframes) specifying their intent, and the AutoPandas engine searches for programs using the Pandas library that transform the input to the output.

## Atlas

[Atlas](https://github.com/rbavishi/atlas) is the framework instantiating AutoPandas. It generalizes the key ideas behind its synthesis engine making it possible to apply them to build engines for entirely new domains such as other APIs like Numpy or Tensorflow, or domain-specific languages (DSLs) for say string-manipulation.

## What are we going to learn?

Pandas is a huge library, for which building a synthesizer can be quite time-consuming. Instead we are going to use the key ideas behind AutoPandas to develop a synthesis engine for **string-manipulation** using Python's `split`, `join`, `slicing` and `indexing` operators for strings and lists. Our engine will accept an input string, and the corresponding output string as the input-output example and produce a Python program using these four operators that performs the desired transformation. In the process, we will also demonstrate the key features of Atlas that can help you get started on building your own synthesis engines.


### 1. The String Manipulation Task

The task is to synthesize string-manipulating programs that use a combination of the `split`, `join`, `slicing` and `indexing` functionalities of Python that transform an input string to the desired output string. As an example, suppose a user wants to write a program to extract the first name from the full name of a person. The full name contains the first name and the last name separated by a space i.e. ` `. The following Python program solves the task. Here `inp` is the variable containing the input string.

```python
inp.split(' ')[0]
```

Try it out!

In [1]:
inp = "Alan Turing"
inp.split(' ')[0]

'Alan'

The `split` function produces a list of strings that are separated by ` ` in the original input string. The `[0]` indexing simply picks the first string.

**Exercise**: Write a program to extract the last name from the full name of a person. Again, assume the first and last names are separated by a space i.e. ` `.

In [2]:
inp = "Alan Turing"
inp.split(' ')[1]

'Turing'

Similarly, we can change the separator used in a string using a combination of `split` and `join`. Such a program can be especially useful for say, changing CSV formats.

In [3]:
inp = "Alan,Turing"
".".join(inp.split(","))

'Alan.Turing'

### 2. Introduction to Generators

Synthesis is fundamentally a search problem. An engine designer defines the space of valid programs and then comes up with a search algorithm to efficiently search over this space. The search space can be trivial in some cases - the set of simple arithmetic programs using variables along with addition, subtraction and multiplication operators can be concisely captured using a context-free grammar. 

The answer is not so simple even for the mini-language we described above using the four string-manipulation operators! The reason is that there are many implicit *constraints* that need to be satisfied by *valid* programs. For example, the string passed to `split` must be present as a substring in the string being splitted on to produce a meaningful result. *Such a constraint cannot be expressed in a grammar.*

The solution used in AutoPandas and Atlas is to use **generators** that are Python functions that can express these constraints as regular code, but at the same time can express all the possibilities. Note that these generators are similar to the generators offered by Python itself, but are far more customizable, the most important of them being that they are **trainable**. Assume that all usage of the term *generator* refers to *our* notion of the generator rather than the Python generator. Nonetheless, let's quickly review our notion of a generator via a simple example.

Let us try writing a generator to produce all *binary strings* of a particular length.

In [4]:
from atlas import generator

@generator
def binary(length: int):
    result = ""
    for bit in range(length):
        result += Select(["0", "1"])
        
    return result

for s in binary.generate(2):
    print(s)

00
01
10
11


Let us break down the moving parts here -

1. `@generator` is a decorator used to mark a function as a generator.
2. The `binary` function takes the length as an argument and uses a loop to construct the string.
3. `binary` uses a special `Select` operator to **non-deterministically** choose the bit (`"0"` or `"1"`).
4. `binary.generate` returns an iterator that explores **all** possible executions of the binary function for the given argument for length.
5. The exploration happens in a *depth-first* manner i.e. the second `Select` returns `"1"` before the first `Select` explores the next value. Hence `"01"` is printed before `"10"`.

### 3. Solving a Simpler Synthesis Problem

Instead of using the concept of generators introduced above to build an engine for entire mini-language we described in the first section, let us solve a simpler problem first. We will only synthesize programs of the following form -

```python
inp.split(<str>)[<int>]
```

That is, we will only produce programs that split the input on a concrete string, and then pick one of the elements produced by this splitting. Let us write a generator express all such programs given an input string.

In [5]:
@generator
def splitting_program(inp: str):
    #  We choose the separator to be a substring of the input string.
    #  We do this by calling the special Substr operator to **non-deterministically* return a substring
    sep: str = Substr(inp)
        
    #  Perform the split so we know how many splits there are.
    splits = inp.split(sep)
    
    #  Now choose a number from 0 to len(splits)
    index = Select(range(0, len(splits)))
    
    #  Construct the program as a string using `sep` and `index`
    return f"inp.split('{sep}')[{index}]"

for prog in splitting_program.generate("Alan Turing"):
    print("Program :", prog)

Program : inp.split('A')[0]
Program : inp.split('A')[1]
Program : inp.split('Al')[0]
Program : inp.split('Al')[1]
Program : inp.split('Ala')[0]
Program : inp.split('Ala')[1]
Program : inp.split('Alan')[0]
Program : inp.split('Alan')[1]
Program : inp.split('Alan ')[0]
Program : inp.split('Alan ')[1]
Program : inp.split('Alan T')[0]
Program : inp.split('Alan T')[1]
Program : inp.split('Alan Tu')[0]
Program : inp.split('Alan Tu')[1]
Program : inp.split('Alan Tur')[0]
Program : inp.split('Alan Tur')[1]
Program : inp.split('Alan Turi')[0]
Program : inp.split('Alan Turi')[1]
Program : inp.split('Alan Turin')[0]
Program : inp.split('Alan Turin')[1]
Program : inp.split('Alan Turing')[0]
Program : inp.split('Alan Turing')[1]
Program : inp.split('l')[0]
Program : inp.split('l')[1]
Program : inp.split('la')[0]
Program : inp.split('la')[1]
Program : inp.split('lan')[0]
Program : inp.split('lan')[1]
Program : inp.split('lan ')[0]
Program : inp.split('lan ')[1]
Program : inp.split('lan T')[0]
Progra

Let's use this generator to build a simple brute-force synthesizer!

In [6]:
def synthesize(inp, out):
    for prog in splitting_program.generate(inp):
        if eval(prog, {'inp': inp}) == out:
            print("Found Solution :", prog)
            
synthesize("Alan Turing", "Turing")

Found Solution : inp.split('Alan ')[1]
Found Solution : inp.split('lan ')[1]
Found Solution : inp.split('an ')[1]
Found Solution : inp.split('n ')[1]
Found Solution : inp.split(' ')[1]


It found the program you wrote in the string manipulation exercise!

```python
inp.split(' ')[1]
```

But only after it found a number of *over-fitting* solutions; solutions that we probably wouldn't write ourselves given what we know about the problem.

```python
inp.split('Alan ')[1]
inp.split('lan ')[1]
inp.split('an ')[1]
inp.split('n ')[1]
```

#### Key Question : Can we *train* our generator to produce the more "*viable*" solutions first?

### 4. Training our Simple Generator

We wish to train our generator, in particular the call to the `Substr` operator, to explore *meaningful* values first. This can be achieved by **backing** the `Substr` operator with a *neural-network* model that looks at the input-output example, and accordingly reorders the possible substrings in the descending order of their probabilities (viability/meaningfulness). But first, we will need to tell the `Substr` operator what the input-output example is. This can be achieved by passing it as a `context`.

In [7]:
@generator
def splitting_program_smart(inp: str, out: str):  # CHANGED : We're taking output as an argument now.
    sep: str = Substr(inp, context=(inp, out))  # CHANGED : Passing the tuple (inp, out) as context
    splits = inp.split(sep) 
    index = Select(range(0, len(splits)))
    
    #  Construct the program as a string using `sep` and `index`
    return f"inp.split('{sep}')[{index}]"

def synthesize_smart(inp, out):
    for prog in splitting_program_smart.generate(inp, out):  # CHANGED : Passing out as an argumen
        if eval(prog, {'inp': inp}) == out:
            print("Found Solution :", prog)

Now let's define a model for our generator. We'll use a graph-neural-network model which encodes characters of a string as nodes in a graph, and has edges to capture both the adjacency between characters within a string, and equality amongst characters. In the interest of time, instead of going into the specifics of the model itself, we'll focus on integrating these models with our generator.

##### TODO : Have a diagram for the graph-encoding described above.

In [8]:
from atlas.operators import operator
from atlas.models.pytorch.imitation import PyTorchGeneratorSharedStateModel
from string_models import SubstrModel

import torch

class SmallGeneratorModel(PyTorchGeneratorSharedStateModel):
    @operator
    def Substr(self, *args, **kwargs):
        return SubstrModel(node_dim=10)
    
    def get_initial_state(self):
        return torch.zeros(1, 10)

The `SmallGeneratorModel` specifies that the Substr operator should use the `SubstrModel` definition. Let's get some training data for our generator.

In [9]:
import pickle
from urllib.request import urlopen
data = pickle.load(urlopen("https://risecamp2019-atlas.s3.us-east-2.amazonaws.com/string_dataset_small.pkl"))

In [10]:
data[0]


        GeneratorTrace(inputs=(('Uzair___Naylor', 'Naylor'), {}),
                       op_traces=[
OpTrace(op_info=OpInfo(sid='/splitting_program_smart/Substr@@1', gen_name='splitting_program_smart', op_type='Substr', index=1, gen_group=None, uid=None, tags=None),
        choice='___',
        domain='Uzair___Naylor',
        context=('Uzair___Naylor', 'Naylor'),
        **{}
       ), 
OpTrace(op_info=OpInfo(sid='/splitting_program_smart/Select@@1', gen_name='splitting_program_smart', op_type='Select', index=1, gen_group=None, uid=None, tags=None),
        choice=1,
        domain=range(0, 2),
        context=None,
        **{}
       )]

The dataset contains *traces* of generators. These are executions of the generator that produce the program we desire. In each case, the `choice` field contains the *correct* choice to be made by each of the special `Substr` and `Select` operators in the generator to produce the correct program. Atlas allows us to *directly* train on these traces.

In [14]:
model = SmallGeneratorModel()
train_data = data[:250]
valid_data = data[250:]
model.train(train_data, valid_data, num_epochs=5)

[Epoch 0] Training Loss: 0.1674 Training Acc: 0.08
[Epoch 0] Validation Loss: 0.0930 Validation Acc: 0.19
[Epoch 1] Training Loss: 0.0586 Training Acc: 0.40
[Epoch 1] Validation Loss: 0.0437 Validation Acc: 0.67
[Epoch 2] Training Loss: 0.0293 Training Acc: 0.76
[Epoch 2] Validation Loss: 0.0137 Validation Acc: 0.96
[Epoch 3] Training Loss: 0.0050 Training Acc: 1.00
[Epoch 3] Validation Loss: 0.0020 Validation Acc: 1.00
[Epoch 4] Training Loss: 0.0010 Training Acc: 1.00
[Epoch 4] Validation Loss: 0.0007 Validation Acc: 1.00


In [15]:
splitting_program_smart.set_default_model(model)
synthesize_smart("Alan  Turing", "Turing")

Found Solution : inp.split('  ')[1]
Found Solution : inp.split(' ')[2]
Found Solution : inp.split(' ')[2]
Found Solution : inp.split('Alan  ')[1]
Found Solution : inp.split('lan  ')[1]


The correct solution was printed first! Since we have trained our generator, it can also work on very large strings, for which brute-force enumeration might be slow to get to the right one.

In [16]:
synthesize_smart("WowSuchALongPrefixCanBlowThingsUpQuiteABit_Alan  Turing", "Turing")

Found Solution : inp.split('  ')[1]
Found Solution : inp.split('_Alan  ')[1]
Found Solution : inp.split(' ')[2]
Found Solution : inp.split(' ')[2]


## Up Next : Building a Synthesizer for the Full Mini-Language

Enough with toy problems! In the next tutorial, we will build a synthesis engine for the full mini-language using `join`, `split`, `slicing` and `indexing`. 